In [ ]:
"""
selenium_spain_adalimumab_scraper_v3.py

- Fills the exact search box input (id contains 'boxSearch') with 'adalimumab'
- Clicks the exact submit button (id contains 'botonSubmitSearch')
- Collects detail-page URLs matching 'adalimumab' (prefers 'detalle_licitacion')
- Pages through results using the input[type=image] next control (id contains 'tableResultSearch' / 'j_id_1g')

Additional behavior:
- For every collected detail_url the script will open the detail page in a new tab,
  try to extract the "file name" displayed on that page (span with id containing 'text_Expediente'
  or adjacent label/value pairs), and include that file name in the output CSV.
  This is useful because the tender id is not always visible in the results table.

Output:
  - collected_adalimumab_urls.csv with columns: file_name, tender_id, detail_url

CONFIG:
  - CHROME_DRIVER_PATH if chromedriver is not on PATH
  - HEADLESS=False to visually debug
"""
import time, re, logging, csv
from urllib.parse import urljoin, urlparse

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, WebDriverException

# ------------- CONFIG -------------
SEARCH_URL = "https://contrataciondelestado.es/wps/portal/plataforma/buscador/!ut/p/z1/jZBNb4MwDIZ_yw69xk4oFI7hoyxTp5WihJLLlKG0ZYKCWrTt549x2WkM32w9zyvboOEI-mo-6rMZ6u5qmrEvtffKVZJxETiIQRQiO_D1IZWMYbCBYgLC3Pd5SDmiI33kXpZFQYwj4IBe4rtOtFZPau_lIkUUj9t4J6mLKfOW-fhHcVzmzwB6Pr4APSFzF_yXUY47bH5_-CKfOYodFVmyZxQjCvlPRtW1pH5ryclU9k767jY0diBKJIWIoVzh12VomxXeq663uTW36qJq-0mmMfStlPKI725_fvgGrHjPHg!!/dz/d5/L2dBISEvZ0FBIS9nQSEh/p0/IZ7_BS88AB1A0OUMA0IL1IQEP210C1=CZ6_AVEQAI93009CB02RA4RGU22097=LA0=Ecom.ibm.faces.portlet.VIEWID!QCPxhtmlQCPscopeSearchView.xhtml==/#Z7_BS88AB1A0OUMA0IL1IQEP210C1"
SEARCH_QUERY = "adalimumab"
HEADLESS = True
CHROME_DRIVER_PATH = None  # set to path if needed
MAX_PAGES = 200
WAIT = 1.0
ADALIMUMAB_PATTERN = re.compile(r"adalimumab", re.I)
OUTPUT_CSV = "collected_adalimumab_urls.csv"

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# ---------------- utility to extract tender id ----------------
def extract_tender_id_from_text(text):
    """
    Heuristic extraction of tender id / file number from visible text:
    - looks for patterns like 2025/006468
    - looks for "Expediente ..." patterns
    - fallback: any token starting with 20xx
    """
    if not text:
        return None
    m = re.search(r"\b(20\d{2}\/[0-9A-Za-z\-\_]{2,40})\b", text)
    if m:
        return re.sub(r"[^\w\-_\.]", "_", m.group(1))
    m2 = re.search(r"(?:Expediente|Expdte|Expediente nº|Expediente n[oº]\.?)\W*[:\-]?\s*([A-Z0-9\/\-\._]{4,60})", text, flags=re.I)
    if m2:
        return re.sub(r"[^\w\-_\.]", "_", m2.group(1))
    m3 = re.search(r"\b(20\d{2}[^\s,;]{2,40})\b", text)
    if m3:
        return re.sub(r"[^\w\-_\.]", "_", m3.group(1))
    return None

# ------------- helpers -------------
def make_driver(headless=HEADLESS):
    """
    Create Chrome webdriver with minimal options. Caller can set CHROME_DRIVER_PATH if necessary.
    """
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,1000")
    if CHROME_DRIVER_PATH:
        driver = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH, options=opts)
    else:
        driver = webdriver.Chrome(options=opts)
    driver.implicitly_wait(5)
    return driver

def ensure_search_box_and_submit(driver, query, wait_seconds=10):
    """
    Find the search input by id containing 'boxSearch', fill it with query,
    and click the submit button id containing 'botonSubmitSearch'.
    Returns True if submission attempted (even if results take time).
    """
    wait = WebDriverWait(driver, wait_seconds)
    try:
        try:
            input_el = driver.find_element(By.XPATH, "//*[contains(@id, 'boxSearch') and (@type='text' or @type='search')]")
        except Exception:
            input_el = driver.find_element(By.CSS_SELECTOR, "input[type='text'], input[type='search']")
        try:
            input_el.clear()
        except Exception:
            pass
        try:
            input_el.send_keys(query)
        except Exception:
            driver.execute_script("arguments[0].value = arguments[1]; arguments[0].dispatchEvent(new Event('input'));", input_el, query)
        time.sleep(0.4)
        try:
            submit_el = driver.find_element(By.XPATH, "//*[contains(@id, 'botonSubmitSearch') and (@type='submit' or @type='button')]")
        except Exception:
            try:
                submit_el = driver.find_element(By.XPATH, "//button[contains(normalize-space(.),'Buscar') or contains(., 'Buscar')]")
            except Exception:
                submit_el = None
        if submit_el:
            driver.execute_script("arguments[0].scrollIntoView(true);", submit_el)
            time.sleep(0.2)
            try:
                submit_el.click()
            except Exception:
                try:
                    input_el.send_keys(Keys.ENTER)
                except Exception:
                    driver.execute_script("arguments[0].click();", submit_el)
            time.sleep(1.0)
            try:
                wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, "table tbody tr")) > 0)
            except Exception:
                pass
            return True
        else:
            logger.warning("Search submit element not found.")
            return False
    except Exception as e:
        logger.exception("ensure_search_box_and_submit failed: %s", e)
        return False

def rows_selector_try(driver):
    """
    Try several selectors to find meaningful result rows. Return list of WebElements.
    """
    selectors = [
        "table tbody tr",
        ".ui-datatable-data tr",
        "#tableResultSearch tbody tr",
        ".searchResults table tbody tr",
    ]
    for sel in selectors:
        try:
            elems = driver.find_elements(By.CSS_SELECTOR, sel)
            meaningful = [e for e in elems if e.text.strip()]
            if meaningful:
                return meaningful
        except Exception:
            continue
    try:
        elems = driver.find_elements(By.TAG_NAME, "tr")
        return [e for e in elems if e.text.strip()]
    except Exception:
        return []

def prefer_detail_link_from_row(row_el, current_page_url):
    """
    From a results row, prefer a link that points to detalle_licitacion (detail page).
    If not present, pick a reasonable fallback (pdf/document).
    """
    anchors = row_el.find_elements(By.TAG_NAME, "a")
    fallback_href = None
    pdf_href = None
    for a in anchors:
        try:
            href = a.get_attribute("href")
            text = (a.text or "").strip()
            if href:
                href = urljoin(current_page_url, href)
                href_l = href.lower()
                if "detalle_licitacion" in href_l or "deeplink:detalle_licitacion" in href_l:
                    return href
                if "detalle de la licitación" in text.lower() or "detalle de la licitaci" in text.lower():
                    return href
                if "getdocumentbyidservlet" in href_l or href_l.endswith(".pdf"):
                    pdf_href = pdf_href or href
                fallback_href = fallback_href or href
        except Exception:
            continue
    if fallback_href:
        return fallback_href
    if pdf_href:
        return pdf_href
    return None

RANGE_RE = re.compile(r"(\d{1,3})\s*-\s*(\d{1,3})\s*(?:de|of)\s*(\d{1,6})", flags=re.I)

def _get_results_range_from_page(driver):
    """
    Parse a visible "start - end of total" text on the page and return tuple (start, end, total) when possible.
    """
    try:
        body_text = driver.find_element(By.TAG_NAME, "body").text
    except Exception:
        body_text = ""
    m = RANGE_RE.search(body_text)
    if not m:
        return None
    try:
        return int(m.group(1)), int(m.group(2)), int(m.group(3))
    except Exception:
        return None

def click_next_exact(driver, timeout=12, retry_if_no_change=True):
    """
    Click the 'Next' control (targets input[type=image] with id suffix ':j_id_1g') and wait until page moves forward.
    Returns True when the page advanced; False otherwise.
    """
    wait = WebDriverWait(driver, timeout)
    before_range = _get_results_range_from_page(driver)
    before_start = before_range[0] if before_range else None
    xpath_next = "//input[@type='image' and contains(@id, 'tableResultSearch') and contains(@id, ':j_id_1g')]"
    try:
        next_btn = driver.find_element(By.XPATH, xpath_next)
    except Exception:
        try:
            cand = driver.find_elements(By.XPATH, "//input[@type='image' and contains(@src,'NextButton.gif')]")
            next_btn = None
            for c in cand:
                try:
                    iid = c.get_attribute("id") or ""
                    if ":j_id_1g" in iid or iid.endswith("j_id_1g"):
                        next_btn = c
                        break
                except Exception:
                    continue
            if next_btn is None and cand:
                for c in cand:
                    try:
                        iid = c.get_attribute("id") or ""
                        if ":j_id_1e" not in iid:
                            next_btn = c
                            break
                    except Exception:
                        continue
        except Exception:
            next_btn = None
    if not next_btn:
        logger.info("Next button (exact) not found by id/xpath.")
        return False
    try:
        driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
    except Exception:
        pass
    def do_click(el):
        try:
            driver.execute_script("arguments[0].click();", el)
        except Exception:
            try:
                el.click()
            except Exception as e:
                logger.debug("Both JS click and .click() failed: %s", e)
                raise
    for attempt in (1, 2) if retry_if_no_change else (1,):
        try:
            do_click(next_btn)
        except Exception as e:
            logger.warning("Clicking Next failed on attempt %d: %s", attempt, e)
            time.sleep(0.5)
            try:
                next_btn = driver.find_element(By.XPATH, xpath_next)
                do_click(next_btn)
            except Exception as e2:
                logger.debug("Re-finding and clicking next failed: %s", e2)
                return False
        try:
            def range_moved(drv):
                new = _get_results_range_from_page(drv)
                if not new:
                    try:
                        rows = drv.find_elements(By.CSS_SELECTOR, "table tbody tr")
                        if rows and len(rows) > 0:
                            first = next((r.text.strip() for r in rows if r.text.strip()), "")
                            return first != ""
                    except Exception:
                        return False
                else:
                    new_start = new[0]
                    if before_start is None:
                        return new_start != (before_start or 1)
                    return new_start > (before_start or 0)
            WebDriverWait(driver, timeout).until(range_moved)
            time.sleep(0.4)
            after_range = _get_results_range_from_page(driver)
            if before_start is not None and after_range is not None:
                if after_range[0] > before_start:
                    logger.info("Successfully advanced to next page: %s -> %s", before_start, after_range[0])
                    return True
                else:
                    logger.warning("After click, start did not increase: before=%s after=%s", before_start, after_range[0])
            else:
                logger.info("Advanced page (couldn't parse numeric ranges, but content updated).")
                return True
        except TimeoutException:
            logger.warning("Timed out waiting for page change after clicking Next (attempt %d).", attempt)
            time.sleep(0.5)
            try:
                next_btn = driver.find_element(By.XPATH, xpath_next)
            except Exception:
                break
    logger.info("Failed to advance to next page after clicking Next.")
    return False

# ------------- new small helper to probe detail page for "file name" -------------
def extract_file_name_from_detail(driver, url, wait_seconds=1.5):
    """
    Open detail URL in a new tab, try to extract the 'file name' shown on the detail page.
    Looks for:
      - span with id containing 'text_Expediente'
      - sibling span following a label span with id containing 'label_Expediente'
      - span whose title/text contains 'Expediente' label nearby
    Returns the found string or None.
    """
    result = None
    original_handle = None
    try:
        original_handle = driver.current_window_handle
    except Exception:
        original_handle = None

    try:
        # open new tab
        driver.execute_script("window.open('');")
        handles = driver.window_handles
        new_handle = handles[-1]
        driver.switch_to.window(new_handle)
        driver.get(url)
        time.sleep(wait_seconds)

        # 1) direct id-based candidate(s)
        try:
            elems = driver.find_elements(By.XPATH, "//*[contains(@id,'text_Expediente')]")
            for e in elems:
                val = (e.get_attribute("title") or e.text or "").strip()
                if val:
                    result = val
                    break
        except Exception:
            pass

        if not result:
            # 2) look for 'label_Expediente' then following sibling span (label/value pattern)
            try:
                labs = driver.find_elements(By.XPATH, "//*[contains(@id,'label_Expediente')]")
                for lab in labs:
                    try:
                        sib = lab.find_element(By.XPATH, "following-sibling::*[1]")
                        val = (sib.get_attribute("title") or sib.text or "").strip()
                        if val:
                            result = val
                            break
                    except Exception:
                        continue
            except Exception:
                pass

        if not result:
            # 3) broader fallback: any span whose title/text looks like an expediente token (slash-separated or contains '/' sequences)
            try:
                spans = driver.find_elements(By.TAG_NAME, "span")
                for s in spans:
                    txt = (s.get_attribute("title") or s.text or "").strip()
                    if txt and re.search(r"\d{2,4}\/.+\/.+", txt):
                        result = txt
                        break
            except Exception:
                pass

        if not result:
            # 4) last resort: look for any element containing the word 'Expediente' nearby and take adjacent token
            try:
                labels = driver.find_elements(By.XPATH, "//*[contains(translate(text(),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'expediente')]")
                for lab in labels:
                    txt = (lab.get_attribute("title") or lab.text or "").strip()
                    if txt and not txt.lower().startswith("expediente"):
                        result = txt
                        break
            except Exception:
                pass

    except Exception as e:
        logger.debug("extract_file_name_from_detail error for %s : %s", url, e)
    finally:
        # close the tab and switch back (if possible)
        try:
            driver.close()
        except Exception:
            pass
        try:
            if original_handle and original_handle in driver.window_handles:
                driver.switch_to.window(original_handle)
            elif driver.window_handles:
                driver.switch_to.window(driver.window_handles[0])
        except Exception:
            pass

    return result

# ------------- main -------------
def collect_detail_urls(search_url, query, max_pages=MAX_PAGES, headless=HEADLESS):
    """
    Main routine:
      - open search page, submit query
      - iterate results pages, collect detail links
      - for each detail link, probe the detail page briefly to extract file_name if present
      - return list of tuples (file_name, tender_id, detail_url)
    """
    driver = make_driver(headless=headless)
    wait = WebDriverWait(driver, 12)
    driver.get(search_url)
    time.sleep(1.5)
    # try to dismiss cookie prompts
    try:
        for txt in ["Aceptar", "Aceptar cookies", "ACEPTAR", "Aceptar las cookies"]:
            try:
                btn = driver.find_element(By.XPATH, f"//button[contains(., '{txt}') or contains(., '{txt.lower()}')]")
                if btn.is_displayed():
                    btn.click()
                    time.sleep(0.5)
            except Exception:
                pass
    except Exception:
        pass

    ok = ensure_search_box_and_submit(driver, query)
    if not ok:
        logger.warning("Search submit did not appear to work (but continuing).")

    collected = []  # list of tuples (file_name, tender_id, detail_url)
    page = 0
    last_first_row_text = None

    while True:
        page += 1
        logger.info("Processing results page %d", page)
        try:
            wait.until(lambda d: len(rows_selector_try(d)) > 0)
        except TimeoutException:
            logger.warning("No rows appeared on page %d (timeout).", page)
            break

        rows = rows_selector_try(driver)
        logger.info("Found %d rows on page %d", len(rows), page)

        current_url = driver.current_url

        for r in rows:
            try:
                row_text = r.text or ""
                if not ADALIMUMAB_PATTERN.search(row_text):
                    continue
                href = prefer_detail_link_from_row(r, current_url)
                if not href:
                    logger.info("Matched row text but no link found; skipping.")
                    continue

                # try to extract tender id from row text or anchor title
                tid = extract_tender_id_from_text(row_text)
                if not tid:
                    try:
                        a_with_title = r.find_element(By.XPATH, ".//a[@title and string-length(normalize-space(@title))>0]")
                        tid = extract_tender_id_from_text(a_with_title.get_attribute("title") or "")
                    except Exception:
                        pass

                # try idEvl param fallback
                if not tid:
                    parsed = urlparse(href)
                    m = re.search(r"idEvl=([^&]+)", parsed.query)
                    if m:
                        tid = re.sub(r"[^\w\-_\.]", "_", m.group(1))

                tid = tid or ""

                # probe detail page to extract file_name (safe: opens new tab then closes it)
                file_name = None
                try:
                    file_name = extract_file_name_from_detail(driver, href, wait_seconds=1.2)
                except Exception as e:
                    logger.debug("extract_file_name_from_detail failed: %s", e)

                file_name = file_name or ""

                record = (file_name, tid, href)
                if record not in collected:
                    collected.append(record)
                    logger.info("Collected: %s  (file_name=%s tender_id=%s)", href, file_name, tid)
            except Exception as e:
                logger.debug("Row parse error: %s", e)

        # capture first-row text for change detection
        try:
            rs2 = rows_selector_try(driver)
            last_first_row_text = rs2[0].text.strip() if rs2 and rs2[0].text else None
        except Exception:
            last_first_row_text = None

        if page >= max_pages:
            logger.info("Reached max pages (%d). Stopping.", max_pages)
            break

        clicked = click_next_exact(driver)
        if not clicked:
            logger.info("Next button not found or not clickable. Stopping pagination.")
            break

        # wait until first row changes (indicating new page)
        try:
            def first_row_changed(drv):
                try:
                    rs = rows_selector_try(drv)
                    if not rs: return False
                    cur = rs[0].text.strip() if rs[0].text else ""
                    return cur != (last_first_row_text or "")
                except Exception:
                    return False
            WebDriverWait(driver, 10).until(first_row_changed)
            time.sleep(0.6)
        except TimeoutException:
            logger.warning("Page didn't change after next click; may be end. Continuing/Breaking.")
            try:
                rsnow = rows_selector_try(driver)
                cur2 = rsnow[0].text.strip() if rsnow and rsnow[0].text else ""
                if cur2 == (last_first_row_text or ""):
                    logger.info("No change detected after clicking next; stopping.")
                    break
            except Exception:
                break

        time.sleep(WAIT)

    driver.quit()
    logger.info("Collected %d URL rows matching '%s'", len(collected), query)
    return collected

if __name__ == "__main__":
    rows = collect_detail_urls(SEARCH_URL, SEARCH_QUERY, max_pages=MAX_PAGES, headless=HEADLESS)
    print("---- Collected detail URLs ----")
    for fname, tid, u in rows:
        print(f"{fname}\t{tid}\t{u}")

    # Save to CSV
    try:
        with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as fh:
            writer = csv.writer(fh)
            writer.writerow(["file_name", "tender_id", "detail_url"])
            for fname, tid, u in rows:
                writer.writerow([fname, tid, u])
        logger.info("Saved collected URLs to %s (%d rows)", OUTPUT_CSV, len(rows))
    except Exception as e:
        logger.exception("Failed to write output CSV %s : %s", OUTPUT_CSV, e)


In [ ]:

"""
Download Adjudicación HTML documents for rows in an input CSV.
Input CSV columns: file_name, tender_id, detail_url

Outputs:
 - downloaded_adjudicacion/   (downloaded html files)
 - download_log.csv           (status log)

This variant SKIPS PDFs (and other non-HTML binary content).
"""

import os
import re
import time
import logging
from pathlib import Path
from urllib.parse import urljoin, unquote, urlparse

import requests
import pandas as pd
from bs4 import BeautifulSoup

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# -----------------------
# Configuration
# -----------------------
CSV_INPUT = Path("collected_adalimumab_urls.csv")    # change if needed
HTML_FOLDER = Path("html_pages")      # optional local saved detail pages, filenames like 2025_010027.html
OUTPUT_FOLDER = Path("downloaded_adjudicacion")
DOWNLOAD_LOG = Path("download_log.csv")
SESSION = requests.Session()
SESSION.headers.update({"User-Agent": "Mozilla/5.0 (compatible; adjudication-downloader/1.1)"})

# If True, re-download and overwrite files even if they exist; otherwise skip existing files
OVERWRITE = False

# polite pause between requests
REQUEST_DELAY = 0.5

# -----------------------
# Helpers
# -----------------------
def load_local_or_fetch_html(local_path: Path, url: str) -> str:
    """Return HTML content from local file if exists, otherwise fetch from url."""
    if local_path and local_path.exists():
        logging.info("Loading local file: %s", local_path)
        return local_path.read_text(encoding="utf-8", errors="ignore")
    # fetch
    logging.info("Fetching detail page: %s", url)
    r = SESSION.get(url, timeout=20)
    r.raise_for_status()
    return r.text

def find_adjudicacion_link(soup: BeautifulSoup) -> str:
    """
    Heuristic search for the Adjudicación link:
    - first look for anchors containing GetDocumentByIdServlet / FileSystem
    - fallback: find text nodes with 'Adjudicación' and find next <a>
    - final fallback: first <a> with an <img alt="Documento html"> or similar
    """
    # 1) direct anchors (most reliable)
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if ("GetDocumentByIdServlet" in href) or ("GetDocumentById" in href) or ("FileSystem" in href):
            context = (a.get_text(" ", strip=True) or "") + " " + (a.find_parent().get_text(" ", strip=True) if a.find_parent() else "")
            if re.search(r"adjudic", context, re.I) or "documento" in context.lower():
                return href

    # 2) find text nodes containing 'Adjudicación' and look for next anchor
    candidates = soup.find_all(text=re.compile(r"Adjudicaci[oó]n|Adjudicacion", re.I))
    for t in candidates:
        el = t.parent
        a = el.find_next("a", href=True)
        if a:
            return a["href"]
        a2 = el.find("a", href=True)
        if a2:
            return a2["href"]

    # 3) image-based anchor (html-icon)
    for a in soup.find_all("a", href=True):
        img = a.find("img")
        if img:
            alt = (img.get("alt") or img.get("title") or "").lower()
            if "documento html" in alt or "html" in alt:
                return a["href"]

    return None

def safe_filename(tender_id: str, original_name: str, ext: str) -> str:
    """Create a safe filename using tender_id + a short original name fragment + extension."""
    orig = re.sub(r'[^A-Za-z0-9_\-\.]', '_', original_name)[:60]
    tid = re.sub(r'[^A-Za-z0-9_\-\.]', '_', tender_id)
    return f"{tid}__{orig}{ext}"

# -----------------------
# Core download routine
# -----------------------
def download_adjudicacion_for_row(file_name: str, tender_id: str, detail_url: str) -> dict:
    result = {
        "file_name": file_name,
        "tender_id": tender_id,
        "detail_url": detail_url,
        "target_url": None,
        "saved_path": None,
        "content_type": None,
        "status": None,
        "error": None
    }
    try:
        # compute local path for detail page if present
        safe_local_name = file_name.replace("/", "_").replace("\\", "_") + ".html"
        local_path = HTML_FOLDER / safe_local_name

        html = load_local_or_fetch_html(local_path, detail_url)
        soup = BeautifulSoup(html, "lxml")

        href = find_adjudicacion_link(soup)
        if not href:
            result["status"] = "no_adjudicacion_link"
            logging.warning("No Adjudicación link found for %s", tender_id)
            return result

        # resolve absolute url
        target_url = href if href.startswith("http") else urljoin(detail_url, href)
        result["target_url"] = target_url
        logging.info("Found adjudicacion target: %s", target_url)

        # request target
        r = SESSION.get(target_url, timeout=30)
        r.raise_for_status()

        ct = (r.headers.get("Content-Type") or "").lower()
        result["content_type"] = ct

        # Explicitly skip PDFs
        if "pdf" in ct or target_url.lower().endswith(".pdf"):
            logging.info("Skipping PDF target for %s (Content-Type=%s)", tender_id, ct)
            result["status"] = "skipped_pdf"
            return result

        # Ensure it's HTML (Content-Type has html or body starts with '<')
        body_start = (r.text or "").lstrip()[:10].lower()
        is_html = ("html" in ct) or (body_start.startswith("<"))
        if not is_html:
            logging.info("Skipping non-HTML target for %s (Content-Type=%s)", tender_id, ct)
            result["status"] = "skipped_non_html"
            return result

        # Save as .html
        parsed = urlparse(target_url)
        orig_fragment = parsed.query or Path(parsed.path).name or "adjudicacion"
        orig_fragment = unquote(orig_fragment)
        ext = ".html"
        fname = safe_filename(tender_id or file_name, orig_fragment, ext)
        OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)
        outpath = OUTPUT_FOLDER / fname

        if outpath.exists() and not OVERWRITE:
            logging.info("File already exists (skip): %s", outpath)
            result["saved_path"] = str(outpath)
            result["status"] = "skipped_exists"
            return result

        # write text (utf-8)
        with open(outpath, "w", encoding="utf-8", errors="ignore") as fh:
            fh.write(r.text)

        result["saved_path"] = str(outpath)
        result["status"] = "downloaded_html"
        logging.info("Saved adjudicacion HTML to %s", outpath)
        return result

    except Exception as e:
        logging.exception("Error downloading adjudicacion for %s", tender_id)
        result["error"] = str(e)
        result["status"] = "error"
        return result

# -----------------------
# Main
# -----------------------
def main():
    if not CSV_INPUT.exists():
        logging.error("Input CSV not found: %s", CSV_INPUT)
        return

    df = pd.read_csv(CSV_INPUT, dtype=str).fillna("")
    logs = []
    for idx, row in df.iterrows():
        file_name = row.get("file_name") or row.get("FileName") or row.get("filename") or ""
        tender_id = row.get("tender_id") or row.get("tenderId") or row.get("tender") or file_name
        detail_url = row.get("detail_url") or row.get("detailUrl") or row.get("url") or ""
        if not detail_url:
            logging.warning("No detail_url for row idx=%s; skipping", idx)
            logs.append({
                "file_name": file_name,
                "tender_id": tender_id,
                "detail_url": detail_url,
                "status": "no_detail_url",
                "error": "no detail_url provided"
            })
            continue
        res = download_adjudicacion_for_row(file_name.strip(), tender_id.strip(), detail_url.strip())
        logs.append(res)
        time.sleep(REQUEST_DELAY)

    pd.DataFrame(logs).to_csv(DOWNLOAD_LOG, index=False, encoding="utf-8-sig")
    logging.info("All done. Log written to %s", DOWNLOAD_LOG)

if __name__ == "__main__":
    main()


In [ ]:
from bs4 import BeautifulSoup, Tag,Doctype
from datetime import datetime

def extract_tender_and_pubdate_from_soup(soup):
    """
    Accepts a BeautifulSoup object or Tag and returns:
      { "tender_id": str or None, "publication_date": "YYYY-MM-DD" or None }

    If a string is passed accidentally, it will be parsed into a BeautifulSoup.
    """
    # allow passing either a BS object/Tag or raw HTML string
    if isinstance(soup, str):
        soup = BeautifulSoup(soup, "html.parser")
    if not isinstance(soup, (BeautifulSoup, Tag)):
        raise TypeError("soup must be a BeautifulSoup or Tag or HTML string")

    h5 = soup.find("h5")
    if not h5:
        return {"tender_id": None, "publication_date": None}

    # --- tender id: prefer <strong> inside <h5>, fallback to text between labels ---
    tender_id = None
    strong = h5.find("strong")
    if strong and strong.get_text(strip=True):
        tender_id = strong.get_text(strip=True)
    else:
        # fallback: text between "Número de Expediente" and "Publicado"
        full = h5.get_text(" ", strip=True)
        if "Número de Expediente" in full and "Publicado" in full:
            between = full.split("Número de Expediente", 1)[1].split("Publicado", 1)[0]
            tender_id = between.strip() or None

    # --- publication date: scan tokens for dd-mm-yyyy or dd/mm/yyyy without using regex ---
    pub_date_iso = None
    full_text = h5.get_text(" ", strip=True)

    for tok in full_text.split():
        t = tok.strip(" ,.;:()[]")
        t_norm = t.replace("/", "-")
        # look for tokens with exactly two separators (e.g. 04-05-2023)
        if t_norm.count("-") == 2:
            parts = t_norm.split("-")
            if all(part.isdigit() for part in parts) and 1 <= len(parts[0]) <= 2 and 1 <= len(parts[1]) <= 2 and len(parts[2]) in (2,4):
                day, month, year = parts
                # try parsing; prefer 4-digit year format
                fmt_candidates = ["%d-%m-%Y", "%d-%m-%y"] if len(year) == 4 else ["%d-%m-%y", "%d-%m-%Y"]
                parsed = None
                for fmt in fmt_candidates:
                    try:
                        parsed = datetime.strptime(f"{day}-{month}-{year}", fmt)
                        break
                    except ValueError:
                        continue
                if parsed:
                    pub_date_iso = parsed.strftime("%Y-%m-%d")
                    break

    return {"tender_id": tender_id, "publication_date": pub_date_iso}

from typing import Optional, Dict, Union
from bs4 import BeautifulSoup, Tag

def extract_issuing_authority(soup_input: Union[BeautifulSoup, Tag, str]) -> Dict[str, Optional[str]]:
    """
    Extract issuing authority info from the "Entidad Adjudicadora" block.

    This version ensures that the "Contacto" block used for phone/fax/email is
    taken from the content *under* the "Entidad Adjudicadora" section (so if there
    are multiple "Contacto" blocks elsewhere, those are ignored).

    Args:
      soup_input: BeautifulSoup / Tag or raw HTML string containing the fragment.

    Returns:
      Dict with keys:
        - issuing_authority_name
        - issuing_authority_type_label
        - issuing_authority_type
        - issuing_authority_website_label
        - issuing_authority_website
        - issuing_authority_phone
        - issuing_authority_fax
        - issuing_authority_email
    """
    # normalize input
    if isinstance(soup_input, str):
        soup = BeautifulSoup(soup_input, "html.parser")
    elif isinstance(soup_input, (BeautifulSoup, Tag)):
        soup = soup_input
    else:
        raise TypeError("soup_input must be BeautifulSoup, Tag or HTML string")

    def text_or_none(el):
        return el.get_text(" ", strip=True) if el else None

    result: Dict[str, Optional[str]] = {
        "issuing_authority_name": None,
        "issuing_authority_type_label": None,
        "issuing_authority_type": None,
        "issuing_authority_website_label": None,
        "issuing_authority_website": None,
        "issuing_authority_phone": None,
        "issuing_authority_fax": None,
        "issuing_authority_email": None,
    }

    # Find the "Entidad Adjudicadora" header and the following <ul>
    ent_h3 = soup.find(lambda t: t.name == "h3" and "Entidad Adjudicadora" in t.get_text())
    ent_ul = ent_h3.find_next_sibling("ul") if ent_h3 else None

    if ent_ul:
        for li in ent_ul.find_all("li", recursive=False):
            # Name: first li contains <strong> inside a div.noremarca
            strong = li.find("strong")
            if strong:
                result["issuing_authority_name"] = text_or_none(strong)
                continue

            # Label / value rows: label in <span>, value either in div.noremarca or after span
            span = li.find("span")
            label = text_or_none(span) if span else None

            # Try to get a href if present (for website)
            a = li.find("a")
            if a and a.has_attr("href"):
                value = a["href"].strip()
            else:
                # prefer div.noremarca
                value_div = li.find("div", class_="noremarca")
                if value_div:
                    value = text_or_none(value_div)
                else:
                    # fallback: li text minus the label
                    full = li.get_text(" ", strip=True)
                    value = full.replace(label, "", 1).strip() if label else full

            if label and "Tipo de Administración" in label:
                result["issuing_authority_type_label"] = label
                result["issuing_authority_type"] = value or None
            elif label and "Sitio Web" in label:
                result["issuing_authority_website_label"] = label
                result["issuing_authority_website"] = value or None

    # ---- Find the Contacto block that is under the Entidad Adjudicadora section ----
    contact_ul = None
    if ent_h3:
        # gather siblings after ent_h3 until the next h3 (boundary)
        nodes = []
        for sib in ent_h3.next_siblings:
            # stop if we hit another h3 (new section)
            if isinstance(sib, Tag) and sib.name == "h3":
                break
            nodes.append(sib)

        # build a small fragment to search inside the Entidad Adjudicadora content
        fragment_html = "".join(str(n) for n in nodes)
        fragment = BeautifulSoup(fragment_html, "html.parser")

        # Prefer an explicit Contacto <h5> within this fragment
        contact_h5 = fragment.find(lambda t: t.name in ("h5", "h3") and "Contacto" in t.get_text())
        contact_ul = contact_h5.find_next_sibling("ul") if contact_h5 else None

        # If there was no explicit h5 Contacto but there's a .rigCol / .leftCol with Contacto inside, try finding ul directly
        if not contact_ul:
            # common pattern: a "rigCol" or "leftCol" section contains contact info
            possible = fragment.find(lambda t: isinstance(t, Tag) and (t.get("class") and ("rigCol" in t.get("class") or "leftCol" in t.get("class"))))
            if possible:
                contact_ul = possible.find("ul")

    # If ent_h3 not found, fall back to global Contacto search (maintain backward compatibility)
    if contact_ul is None:
        contact_h5_global = soup.find(lambda t: t.name in ("h5", "h3") and "Contacto" in t.get_text())
        contact_ul = contact_h5_global.find_next_sibling("ul") if contact_h5_global else None

    # Extract phone/fax/email from the located contact_ul
    if contact_ul:
        for li in contact_ul.find_all("li", recursive=False):
            span = li.find("span")
            label = text_or_none(span) if span else None

            value_div = li.find("div", class_="noremarca")
            if value_div:
                value = text_or_none(value_div)
            else:
                full = li.get_text(" ", strip=True)
                value = full.replace(label, "", 1).strip() if label else full

            if label and "Teléfono" in label:
                result["issuing_authority_phone"] = value or None
            elif label and "Fax" in label:
                result["issuing_authority_fax"] = value or None
            elif label and "Correo Electrónico" in label:
                result["issuing_authority_email"] = value or None

    return result

# Example usage (comment out when embedding):
# soup = BeautifulSoup(html_fragment, "html.parser")
# print(extract_issuing_authority(soup))

from typing import Optional, Dict, Union
from bs4 import BeautifulSoup, Tag
from datetime import datetime, timedelta, date
import calendar
import re

def extract_contract_details(soup_input: Union[BeautifulSoup, Tag, str]) -> Dict[str, Optional[str]]:
    """
    Extract contract details from the 'Objeto del Contrato' block.

    Returns a dict with keys:
      - tender_title
      - estimated_contract_value
      - cpv_classification    (list of CPV code strings, or None)
      - contract_duration     (exact string as found under "Plazo de Ejecución", or None)

    NOTE: contract_from and contract_till removed as requested.
    """
    # --- normalize input ---
    if isinstance(soup_input, str):
        soup = BeautifulSoup(soup_input, "html.parser")
    elif isinstance(soup_input, (BeautifulSoup, Tag)):
        soup = soup_input
    else:
        raise TypeError("soup_input must be BeautifulSoup, Tag or an HTML string")

    def text_or_none(el):
        return el.get_text(" ", strip=True) if el else None

    out = {
        "tender_title": None,
        "estimated_contract_value": None,
        "cpv_classification": None,   # will be a list of codes when present
        "contract_duration": None,
    }

    # --- 1) Tender title ---
    h2 = soup.find(lambda t: t.name == "h2" and "Objeto del Contrato" in t.get_text())
    if h2:
        dd = h2.find("div", class_="noremarca")
        if dd and dd.get_text(strip=True):
            out["tender_title"] = dd.get_text(" ", strip=True)

    # --- 2) Estimated contract value ---
    val_li = soup.find(lambda t: t.name == "li" and t.find("span") and "Valor estimado del contrato" in t.find("span").get_text())
    if val_li:
        val_div = val_li.find("div", class_="noremarca")
        if val_div and val_div.get_text(strip=True):
            out["estimated_contract_value"] = val_div.get_text(" ", strip=True)
        else:
            span = val_li.find("span")
            full = val_li.get_text(" ", strip=True)
            label = span.get_text(" ", strip=True) if span else ""
            val = full.replace(label, "", 1).strip()
            out["estimated_contract_value"] = val if val else None

    # --- 3) CPV classification: extract only codes from the Objeto del Contrato container ---
    cpv_codes = []
    if h2:
        container = h2.find_parent("div", class_="boxWithBackground") or h2.find_parent("div", class_="box01") or h2.parent
        if container:
            candidates = []
            cpv_span = container.find(lambda t: t.name == "span" and "Clasificación CPV" in t.get_text())
            if cpv_span:
                node = cpv_span.parent
                ul = node.find_next_sibling("ul") or node.find_next("ul")
                if ul:
                    for div in ul.find_all("div", class_="noremarca"):
                        txt = div.get_text(" ", strip=True)
                        if txt:
                            candidates.append(txt)
                else:
                    for div in container.find_all("div", class_="noremarca"):
                        txt = div.get_text(" ", strip=True)
                        if txt:
                            candidates.append(txt)
            else:
                for div in container.find_all("div", class_="noremarca"):
                    txt = div.get_text(" ", strip=True)
                    if txt:
                        candidates.append(txt)

            seen = set()
            for c in candidates:
                found = re.findall(r'\b\d{8}\b', c)
                if not found:
                    found = re.findall(r'\b\d{6,8}\b', c)
                for code in found:
                    if code not in seen:
                        cpv_codes.append(code)
                        seen.add(code)

    out["cpv_classification"] = cpv_codes if cpv_codes else None

    # --- 4) Contract duration: copy the exact string under "Plazo de Ejecución" ---
    contract_duration_raw = None
    # operate within the same container used above (nearest Objeto del Contrato)
    if h2:
        container = h2.find_parent("div", class_="boxWithBackground") or h2.find_parent("div", class_="box01") or h2.parent
    else:
        container = soup

    # look for a span containing "Plazo de Ejecución" (allow accent/no-accent)
    plazo_span = None
    if container:
        # check common exact text
        plazo_span = container.find(lambda t: t.name == "span" and "Plazo de Ejecución" in t.get_text())
        if not plazo_span:
            # try without accent (some variants)
            plazo_span = container.find(lambda t: t.name == "span" and "Plazo de Ejecucion" in t.get_text())
        if not plazo_span:
            # try a shorter fingerprint
            plazo_span = container.find(lambda t: t.name == "span" and "Plazo" in t.get_text() and "Ejec" in t.get_text())

    if plazo_span:
        # usual structure: span -> parent li -> following ul with li/div.noremarca holding the duration
        parent_li = plazo_span.find_parent("li")
        candidates = []
        if parent_li:
            # direct divs inside the li
            for div in parent_li.find_all("div", class_="noremarca", recursive=True):
                txt = div.get_text(" ", strip=True)
                if txt:
                    candidates.append(txt)
            # also check the next sibling UL if present
            nxt_ul = parent_li.find_next_sibling("ul")
            if nxt_ul:
                for div in nxt_ul.find_all("div", class_="noremarca"):
                    txt = div.get_text(" ", strip=True)
                    if txt:
                        candidates.append(txt)
            # sometimes divs are after the li as siblings (example HTML)
            for sib in parent_li.find_next_siblings():
                if isinstance(sib, Tag) and sib.name in ("li", "h4", "h5", "h3"):
                    break
                if isinstance(sib, Tag):
                    for div in sib.find_all("div", class_="noremarca"):
                        txt = div.get_text(" ", strip=True)
                        if txt:
                            candidates.append(txt)
        else:
            # fallback: look in next siblings of the span
            for node in plazo_span.next_siblings:
                if isinstance(node, Tag):
                    for div in node.find_all("div", class_="noremarca"):
                        txt = div.get_text(" ", strip=True)
                        if txt:
                            candidates.append(txt)

        # pick the first candidate that looks like a duration (contains digits and a letter)
        for c in candidates:
            if c and re.search(r'\d', c):
                # strip punctuation/newlines
                cleaned = c.strip()
                # if the string contains a leading ":" or similar, remove it
                cleaned = re.sub(r'^[\:\-\s]+', '', cleaned)
                contract_duration_raw = cleaned
                break

    # If still not found, fallback: search container for div.noremarca matching patterns like 'NNN Día' etc.
    if not contract_duration_raw and container:
        for div in container.find_all("div", class_="noremarca"):
            txt = div.get_text(" ", strip=True)
            if txt and re.search(r'\b\d+\b', txt) and re.search(r'(día|día\(s\)|día\(s\)|Día|Día\(s\)|día|mes|año|día\(s\))', txt, flags=re.IGNORECASE):
                contract_duration_raw = txt.strip()
                break

    # final assign (exact string found or None)
    out["contract_duration"] = contract_duration_raw or None

    return out


from typing import Optional, Dict, Union
from bs4 import BeautifulSoup, Tag
from datetime import datetime
import re

def extract_award_details(soup_input: Union[BeautifulSoup, Tag, str]) -> Dict[str, Optional[str]]:
    """
    Extract award/winner details from the 'Adjudicatario' / award block.

    Returns:
      {
        "winning_company": str or None,
        "winning_company_nif": str or None,
        "final_contract_value": str or None,  # value text from "Importe total ofertado (sin impuestos)"
        "motivation": str or None,
        "award_date": "YYYY-MM-DD" or None,
        "award_date_raw": str or None
      }
    """
    # --- normalize input ---
    if isinstance(soup_input, str):
        soup = BeautifulSoup(soup_input, "html.parser")
    elif isinstance(soup_input, (BeautifulSoup, Tag)):
        soup = soup_input
    else:
        raise TypeError("soup_input must be BeautifulSoup, Tag or HTML string")

    def text_or_none(el):
        return el.get_text(" ", strip=True) if el else None

    out: Dict[str, Optional[str]] = {
        "winning_company": None,
        "winning_company_nif": None,
        "final_contract_value": None,
        "motivation": None,
        "award_date": None,
        "award_date_raw": None,
    }

    # find the Adjudicatario block (h4 "Adjudicatario" inside a boxWithBackground)
    adjud_h4 = soup.find(lambda t: t.name == "h4" and "Adjudicatario" in t.get_text())
    container = adjud_h4.find_parent("div", class_="boxWithBackground") if adjud_h4 else None
    if not container:
        # fallback: search any boxWithBackground that contains an h4 with "Adjudicatario"
        container = soup.find("div", class_="boxWithBackground")

    # 1) Winning company: first <strong> inside this container
    if container:
        strong = container.find("strong")
        if strong:
            out["winning_company"] = text_or_none(strong)

    # 2) NIF: find li where span == "NIF"
    nif_li = container.find(lambda t: t.name == "li" and t.find("span") and "NIF" in t.find("span").get_text()) if container else None
    if nif_li:
        # value usually in div.noremarca
        val = nif_li.find("div", class_="noremarca")
        out["winning_company_nif"] = text_or_none(val) if val else None

    # 3) Final contract value: "Importe total ofertado (sin impuestos)"
    val_li = container.find(lambda t: t.name == "li" and t.find("span") and "Importe total ofertado (sin impuestos)" in t.find("span").get_text()) if container else None
    if val_li:
        vdiv = val_li.find("div", class_="noremarca")
        if vdiv:
            vtxt = text_or_none(vdiv)
            # tidy: remove trailing dots and extra whitespace
            if vtxt:
                vtxt = vtxt.rstrip(". ").strip()
            out["final_contract_value"] = vtxt or None

    # 4) Motivation: find the "Motivación" h5 or the li with span "Motivación"
    motiv_h5 = container.find(lambda t: t.name == "h5" and "Motivación" in t.get_text()) if container else None
    motiv_li = None
    if motiv_h5:
        motiv_block = motiv_h5.find_next_sibling("ul")
        if motiv_block:
            motiv_li = motiv_block.find(lambda t: t.name == "li" and t.find("span") and "Motivación" in t.find("span").get_text())
    else:
        # fallback: search container for a li span "Motivación"
        motiv_li = container.find(lambda t: t.name == "li" and t.find("span") and "Motivación" in t.find("span").get_text()) if container else None

    if motiv_li:
        v = motiv_li.find("div", class_="noremarca")
        out["motivation"] = text_or_none(v) if v else None

    # 5) Award date: find the li with span "Fecha del Acuerdo"
    date_li = container.find(lambda t: t.name == "li" and t.find("span") and "Fecha del Acuerdo" in t.find("span").get_text()) if container else None
    if date_li:
        ddiv = date_li.find("div", class_="noremarca")
        raw = text_or_none(ddiv) if ddiv else None
        if raw:
            # try to find a date token like DD/MM/YYYY or DD-MM-YYYY
            m = re.search(r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})', raw)
            if m:
                tok = m.group(1)
                parsed = None
                for fmt in ("%d/%m/%Y", "%d-%m-%Y", "%d/%m/%y", "%d-%m-%y"):
                    try:
                        parsed_dt = datetime.strptime(tok, fmt)
                        parsed = parsed_dt.date()
                        break
                    except ValueError:
                        continue
                if parsed:
                    out["award_date"] = parsed.isoformat()
                    out["award_date_raw"] = tok
                else:
                    out["award_date_raw"] = raw
            else:
                out["award_date_raw"] = raw

    return out

# -----------------------
# Example usage:
# html = """ ... your fragment ... """
# from bs4 import BeautifulSoup
# soup = BeautifulSoup(html, "html.parser")
# print(extract_award_details(soup))

# --- Example usage ---
if __name__ == "__main__":
    import glob
    import os
    import csv
    from bs4 import BeautifulSoup
    from deep_translator import GoogleTranslator
    import pandas as pd
    import re
    from datetime import datetime

    INPUT_DIR = "downloaded_adjudicacion"
    GLOB_PATTERN = os.path.join(INPUT_DIR, "*.html")
    OUTPUT_CSV = "tender_data_final.csv"

    # translator instance (re-used)
    translator = GoogleTranslator(source="auto", target="en")

    # helper: decide whether to translate a value
    date_re = re.compile(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b')
    currency_re = re.compile(r'^[\s\d\.,]+(?:EUR|€|USD|€\.)?$', re.IGNORECASE)
    url_re = re.compile(r'https?://', re.IGNORECASE)
    nif_re = re.compile(r'^[A-Z]\d{7,}$', re.IGNORECASE)  # simple NIF like A08130502
    code_re = re.compile(r'^\d{4,8}$')  # CPV-like single code

    def should_translate(val: str) -> bool:
        if not val or not isinstance(val, str):
            return False
        s = val.strip()
        if url_re.search(s):
            return False
        if date_re.search(s) and len(s) <= 20:
            return False
        if currency_re.match(s):
            return False
        if nif_re.match(s):
            return False
        if re.fullmatch(r'[\d\.,\s\-\:\(\)]+', s):
            return False
        if code_re.match(s):
            return False
        return True

    # fields/column order: union of keys from all extractors + file name
    columns = [
        "source_file",
        # from tender_and_pubdate
        "tender_id", "publication_date",
        # from issuing authority
        "issuing_authority_name", "issuing_authority_type_label", "issuing_authority_type",
        "issuing_authority_website_label", "issuing_authority_website",
        "issuing_authority_phone", "issuing_authority_fax", "issuing_authority_email",
        # from contract_details (per contract/lot)
        "tender_title", "estimated_contract_value", "cpv_classification", "contract_duration",
        # from award_details (per adjudicatario)
        "winning_company", "winning_company_nif", "final_contract_value",
        "motivation", "award_date", "award_date_raw",
    ]

    records = []

    files = sorted(glob.glob(GLOB_PATTERN))
    if not files:
        print("No HTML files found in", INPUT_DIR)

    # progress-aware processing over files
    try:
        from tqdm import tqdm
        files_iter = tqdm(files, desc="Processing HTML files", unit="file")
        use_tqdm = True
    except Exception:
        files_iter = enumerate(files, 1)
        use_tqdm = False
        total = len(files)

    for it in files_iter:
        if use_tqdm:
            filepath = it
        else:
            idx, filepath = it
            print(f"Processing {idx}/{total}: {os.path.basename(filepath)}")

        try:
            with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
                soup = BeautifulSoup(f, "html.parser")
        except Exception as e:
            print(f"Failed to open/parse {filepath}: {e}")
            continue

        # 1) Global/exact-once extractors
        try:
            tender_global = extract_tender_and_pubdate_from_soup(soup) or {}
        except Exception as e:
            print(f"extract_tender_and_pubdate_from_soup failed for {filepath}: {e}")
            tender_global = {}

        try:
            issuing_global = extract_issuing_authority(soup) or {}
        except Exception as e:
            print(f"extract_issuing_authority failed for {filepath}: {e}")
            issuing_global = {}

        # 2) Find all contract containers (those containing an h2 "Objeto del Contrato")
        contract_containers = []
        for tag in soup.find_all():
            if tag.name and tag.find(lambda t: t.name == "h2" and "Objeto del Contrato" in t.get_text()):
                # choose the highest-level container that encloses the h2: prefer div.boxWithBackground or div.box01
                # the tag itself may be the h2's parent; promote to nearest ancestor with class boxWithBackground/box01
                h2 = tag.find(lambda t: t.name == "h2" and "Objeto del Contrato" in t.get_text())
                container = h2.find_parent("div", class_="boxWithBackground") or h2.find_parent("div", class_="box01") or h2.parent
                # store container only once
                if container and container not in contract_containers:
                    contract_containers.append(container)

        # 3) Find all adjudicatario containers (those containing an h4 "Adjudicatario")
        adjud_containers = []
        for tag in soup.find_all():
            if tag.name and tag.find(lambda t: t.name == "h4" and "Adjudicatario" in t.get_text()):
                h4 = tag.find(lambda t: t.name == "h4" and "Adjudicatario" in t.get_text())
                container = h4.find_parent("div", class_="boxWithBackground") or h4.parent
                if container and container not in adjud_containers:
                    adjud_containers.append(container)

        # Build document order index mapping: enumerate all tags and map id(tag) -> index
        doc_tags = list(soup.find_all())
        tag_index = {id(t): i for i, t in enumerate(doc_tags)}

        # Helper: get index of a container (use first child tag if container not in doc_tags)
        def idx_of(tag):
            return tag_index.get(id(tag), None)

        # If no contract containers found, try fallback: global contract area (use entire soup as single contract)
        if not contract_containers:
            contract_containers = [soup]

        # For each adjud container, find the nearest preceding contract container by document index
        for adjud in adjud_containers:
            adjud_idx = idx_of(adjud)
            # find candidate contract with index < adjud_idx and the largest such index
            chosen_contract = None
            best_idx = -1
            for c in contract_containers:
                c_idx = idx_of(c)
                if c_idx is None:
                    # try to find index via first child
                    children = list(c.find_all())
                    if children:
                        c_idx = idx_of(children[0])
                if c_idx is None:
                    continue
                if adjud_idx is not None and c_idx <= adjud_idx and c_idx > best_idx:
                    best_idx = c_idx
                    chosen_contract = c
            # If not found, fallback to the first contract container
            if chosen_contract is None:
                chosen_contract = contract_containers[0] if contract_containers else soup

            # Extract per-contract details from chosen_contract
            try:
                contract_details = extract_contract_details(chosen_contract) or {}
            except Exception as e:
                print(f"extract_contract_details failed for contract near {filepath}: {e}")
                contract_details = {}

            # Extract award details from adjud container (per adjud)
            try:
                award_details = extract_award_details(adjud) or {}
            except Exception as e:
                print(f"extract_award_details failed for adjud near {filepath}: {e}")
                award_details = {}

            # Build row merging global + contract + award
            row = {k: None for k in columns}
            row["source_file"] = os.path.basename(filepath)

            # helper that will translate text values when appropriate
            def maybe_translate_and_assign(field_name, value):
                if value is None:
                    row[field_name] = None
                    return
                # for CPV list, keep as-is (don't translate numeric codes), join into comma string
                if field_name == "cpv_classification":
                    if isinstance(value, list):
                        row[field_name] = ", ".join(value) if value else None
                    else:
                        row[field_name] = value
                    return
                # do not translate website field
                if field_name in ("issuing_authority_website",):
                    row[field_name] = value
                    return
                # If value is non-string (e.g., list), coerce to str
                if not isinstance(value, str):
                    value = str(value)
                # decide translation
                if should_translate(value):
                    try:
                        translated = translator.translate(value)
                        row[field_name] = translated if translated else value
                    except Exception:
                        row[field_name] = value
                else:
                    row[field_name] = value

            # tender global
            maybe_translate_and_assign("tender_id", tender_global.get("tender_id"))
            row["publication_date"] = tender_global.get("publication_date")

            # issuing authority (global)
            maybe_translate_and_assign("issuing_authority_name", issuing_global.get("issuing_authority_name"))
            maybe_translate_and_assign("issuing_authority_type_label", issuing_global.get("issuing_authority_type_label"))
            maybe_translate_and_assign("issuing_authority_type", issuing_global.get("issuing_authority_type"))
            row["issuing_authority_website_label"] = issuing_global.get("issuing_authority_website_label")
            row["issuing_authority_website"] = issuing_global.get("issuing_authority_website")
            row["issuing_authority_phone"] = issuing_global.get("issuing_authority_phone")
            row["issuing_authority_fax"] = issuing_global.get("issuing_authority_fax")
            row["issuing_authority_email"] = issuing_global.get("issuing_authority_email")

            # contract details (from chosen_contract)
            maybe_translate_and_assign("tender_title", contract_details.get("tender_title"))
            row["estimated_contract_value"] = contract_details.get("estimated_contract_value")
            maybe_translate_and_assign("cpv_classification", contract_details.get("cpv_classification"))
            row["contract_duration"] = contract_details.get("contract_duration")

            # award details (from adjud)
            maybe_translate_and_assign("winning_company", award_details.get("winning_company"))
            row["winning_company_nif"] = award_details.get("winning_company_nif")
            row["final_contract_value"] = award_details.get("final_contract_value")
            maybe_translate_and_assign("motivation", award_details.get("motivation"))
            row["award_date"] = award_details.get("award_date")
            row["award_date_raw"] = award_details.get("award_date_raw")

            records.append(row)

        # If there were contracts but no adjudicatario blocks (rare), create one row per contract using contract details only
        if contract_containers and not adjud_containers:
            for c in contract_containers:
                try:
                    contract_details = extract_contract_details(c) or {}
                except Exception as e:
                    contract_details = {}
                row = {k: None for k in columns}
                row["source_file"] = os.path.basename(filepath)
                maybe_translate_and_assign = lambda fn, val: row.__setitem__(fn, val) if val is not None else None
                # global tender/issuing
                row["tender_id"] = tender_global.get("tender_id")
                row["publication_date"] = tender_global.get("publication_date")
                row["issuing_authority_name"] = issuing_global.get("issuing_authority_name")
                row["issuing_authority_website"] = issuing_global.get("issuing_authority_website")
                # contract fields
                row["tender_title"] = contract_details.get("tender_title")
                row["estimated_contract_value"] = contract_details.get("estimated_contract_value")
                if isinstance(contract_details.get("cpv_classification"), list):
                    row["cpv_classification"] = ", ".join(contract_details.get("cpv_classification"))
                else:
                    row["cpv_classification"] = contract_details.get("cpv_classification")
                row["contract_duration"] = contract_details.get("contract_duration")
                records.append(row)

    # write out to CSV using pandas to preserve headers and quoting
    if records:
        df = pd.DataFrame(records, columns=columns)
        df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
        print(f"Wrote {len(records)} rows to {OUTPUT_CSV}")
    else:
        print("No records extracted.")


In [ ]:
# data_cleaning_with_progress.py
import re
import math
import pandas as pd
from deep_translator import GoogleTranslator
from typing import Optional

INPUT_CSV = "tender_data_final.csv"
OUTPUT_CSV = "tender_data_final_cleaned.csv"

# Columns to keep (in this order)
KEEP_COLS = [
    "tender_id",
    "publication_date",
    "issuing_authority_name",
    "issuing_authority_type",
    "issuing_authority_website",
    "issuing_authority_phone",
    "issuing_authority_fax",
    "issuing_authority_email",
    "tender_title",
    "estimated_contract_value",
    "cpv_classification",
    "contract_duration",
    "winning_company",
    "winning_company_nif",
    "final_contract_value",
    "motivation"
]

# translator (reused)
translator = GoogleTranslator(source="auto", target="en")

# Heuristics to skip translation for fields that are not natural language
_url_re = re.compile(r'https?://', re.I)
_date_re = re.compile(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b')
_currency_like_re = re.compile(r'^[\s\d\.,]+(?:EUR|€|USD|€\.)?$', re.I)
_nif_re = re.compile(r'^[A-Z]\d{7,}$', re.I)
_code_re = re.compile(r'^[\d,\s\.\-]+$')

def should_translate(val: str) -> bool:
    if val is None:
        return False
    if not isinstance(val, str):
        return False
    s = val.strip()
    if not s:
        return False
    if _url_re.search(s):
        return False
    if _date_re.search(s) and len(s) <= 20:
        return False
    if _currency_like_re.match(s):
        return False
    if _nif_re.match(s):
        return False
    if len(s) <= 4 and s.isupper():
        return False
    if _code_re.fullmatch(s):
        return False
    return True

# -------------------------
# Robust currency parsing utilities (European-aware)
def parse_currency_to_float(s: Optional[str]) -> Optional[float]:
    """
    Parse a currency-like string into a float (EUR), with European-format handling.
    Examples:
      - "150.457,5 EUR" -> 150457.5
      - "22.078,8 EUR." -> 22078.8
      - "22,078.8 EUR" -> heuristically handled
      - "22509 EUR" -> 22509.0
    """
    if s is None:
        return None
    s = str(s).strip()
    if not s:
        return None

    # keep only digits, commas, dots and minus sign
    s_clean = re.sub(r'[^0-9\-,\.]', '', s)
    if not s_clean:
        return None

    # If both '.' and ',' present: treat '.' as thousands, ',' as decimal
    if '.' in s_clean and ',' in s_clean:
        # remove thousands dots, convert comma decimal to dot
        s_num = s_clean.replace('.', '').replace(',', '.')
    elif ',' in s_clean and '.' not in s_clean:
        # only comma present: treat comma as decimal separator
        s_num = s_clean.replace(',', '.')
    elif '.' in s_clean and ',' not in s_clean:
        # only dot present: ambiguous
        # If there's exactly one dot and exactly 3 digits after it, assume thousands (remove the dot)
        parts = s_clean.split('.')
        if len(parts) == 2 and len(parts[1]) == 3:
            s_num = ''.join(parts)
        else:
            # otherwise treat dot as decimal
            s_num = s_clean
    else:
        s_num = s_clean

    # If multiple dots remain, keep last as decimal and remove earlier dots
    if s_num.count('.') > 1:
        parts = s_num.split('.')
        s_num = ''.join(parts[:-1]) + '.' + parts[-1]

    # Remove leading/trailing stray characters
    s_num = re.sub(r'^[^0-9\.\-]+|[^0-9\.\-]+$', '', s_num)
    if not s_num:
        return None

    try:
        val = float(s_num)
        return val
    except Exception:
        digits = re.sub(r'[^\d\-]', '', s_num)
        if digits:
            try:
                return float(digits)
            except Exception:
                return None
        return None

# -------------------------
def clean_dataframe_with_progress(df: pd.DataFrame) -> pd.DataFrame:
    # keep only columns that exist and the required ones
    present_cols = [c for c in KEEP_COLS if c in df.columns]
    df = df[present_cols].copy()

    # 1) Normalize tender_title: remove leading colon ":" and surrounding spaces
    if "tender_title" in df.columns:
        df["tender_title"] = df["tender_title"].astype(str).fillna("")
        df["tender_title"] = df["tender_title"].str.replace(r'^\s*[:：]\s*', '', regex=True)
        df["tender_title"] = df["tender_title"].replace({'': None})

    # Prepare output rows
    output_rows = []
    nrows = len(df)

    # Setup progress bar (tqdm if available)
    try:
        from tqdm import tqdm
        iterator = tqdm(range(nrows), desc="Cleaning rows", unit="row")
    except Exception:
        iterator = range(nrows)

    # Build a translation cache to avoid repeated network calls
    translation_cache = {}

    text_columns = [
        "issuing_authority_name", "issuing_authority_type",
        "tender_title", "cpv_classification", "contract_duration",
        "winning_company", "motivation"
    ]

    for i in iterator:
        row = df.iloc[i]
        cleaned_row = {}

        # Copy simple fields directly
        for col in [
            "tender_id", "publication_date",
            "issuing_authority_website", "issuing_authority_phone",
            "issuing_authority_fax", "issuing_authority_email",
            "winning_company_nif", "estimated_contract_value", "final_contract_value",
            "cpv_classification"
        ]:
            if col in row.index:
                cleaned_row[col] = row[col] if pd.notna(row[col]) and row[col] != "" else None
            else:
                cleaned_row[col] = None

        # Textual fields: translate if needed (with cache)
        for col in text_columns:
            if col in row.index:
                val = row[col]
                if val is None or (isinstance(val, float) and math.isnan(val)):
                    cleaned_row[col] = None
                    continue
                val = str(val).strip()
                if val == "":
                    cleaned_row[col] = None
                    continue
                # translation cache
                if val in translation_cache:
                    cleaned_row[col] = translation_cache[val]
                    continue
                # decide whether to translate
                if should_translate(val):
                    try:
                        translated = translator.translate(val)
                        translated = translated if translated else val
                    except Exception:
                        translated = val
                else:
                    translated = val
                translation_cache[val] = translated
                cleaned_row[col] = translated
            else:
                cleaned_row[col] = None

        # Parse currency numeric values per-row (use European-aware parser)
        est_val_raw = row.get("estimated_contract_value") if "estimated_contract_value" in row.index else None
        fin_val_raw = row.get("final_contract_value") if "final_contract_value" in row.index else None
        est_num = parse_currency_to_float(est_val_raw) if est_val_raw not in (None, "") else None
        fin_num = parse_currency_to_float(fin_val_raw) if fin_val_raw not in (None, "") else None

        # contract_value_raw: numeric max of est and final (float), prefer final if equal
        contract_value_raw = None
        if est_num is None and fin_num is None:
            contract_value_raw = None
        elif est_num is None:
            contract_value_raw = fin_num
        elif fin_num is None:
            contract_value_raw = est_num
        else:
            # choose maximum
            contract_value_raw = max(est_num, fin_num)

        cleaned_row["contract_value_raw"] = contract_value_raw  # numeric float or None

        # Keep contract_duration exactly as the translated/cleaned text above (it was translated earlier)
        # contract_duration should be in cleaned_row due to text_columns translation step
        # Ensure contract_duration is None when empty
        if cleaned_row.get("contract_duration") in ("", None):
            cleaned_row["contract_duration"] = None

        # Final assemble: map to final column names required
        out_row = {
            "tender_id": cleaned_row.get("tender_id"),
            "publication_date": cleaned_row.get("publication_date"),
            "issuing_authority_name": cleaned_row.get("issuing_authority_name"),
            "issuing_authority_type": cleaned_row.get("issuing_authority_type"),
            "issuing_authority_website": cleaned_row.get("issuing_authority_website"),
            "issuing_authority_phone": cleaned_row.get("issuing_authority_phone"),
            "issuing_authority_fax": cleaned_row.get("issuing_authority_fax"),
            "issuing_authority_email": cleaned_row.get("issuing_authority_email"),
            "tender_title": cleaned_row.get("tender_title"),
            "estimated_contract_value": cleaned_row.get("estimated_contract_value"),
            "cpv_classification": cleaned_row.get("cpv_classification"),
            "contract_duration": cleaned_row.get("contract_duration"),
            "winning_company": cleaned_row.get("winning_company"),
            "winning_company_nif": cleaned_row.get("winning_company_nif"),
            "final_contract_value": cleaned_row.get("final_contract_value"),
            "motivation": cleaned_row.get("motivation"),
            # numeric float field requested
            "contract_value_raw_EUR": cleaned_row.get("contract_value_raw"),
        }

        output_rows.append(out_row)

    # Build final DataFrame
    df_final = pd.DataFrame(output_rows, columns=[
        "tender_id",
        "publication_date",
        "issuing_authority_name",
        "issuing_authority_type",
        "issuing_authority_website",
        "issuing_authority_phone",
        "issuing_authority_fax",
        "issuing_authority_email",
        "tender_title",
        "estimated_contract_value",
        "cpv_classification",
        "contract_duration",
        "winning_company",
        "winning_company_nif",
        "final_contract_value",
        "motivation",
        "contract_value_raw_EUR"
    ])

    return df_final

# -------------------------
def main():
    print("Loading:", INPUT_CSV)
    df = pd.read_csv(INPUT_CSV, dtype=str, keep_default_na=False, na_values=[""])
    # convert empty strings to None
    df = df.where(df.notnull() & df.astype(bool), None)

    cleaned = clean_dataframe_with_progress(df)
    cleaned.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
    print("Wrote cleaned data to", OUTPUT_CSV)
    print("Rows:", len(cleaned))
    print(cleaned.head(5).to_dict(orient="records"))

if __name__ == "__main__":
    main()
